从sci-hub爬取相关文章并保存为pdf。

In [19]:
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup
ua = UserAgent()

In [20]:
domain_scihub = 'https://sci-hub.tf/'
doi = 'https://doi.org/10.1093/applin/amz034'
sci_url = domain_scihub + doi
headers = {'User-Agent': ua.random}
bs = BeautifulSoup(requests.get(sci_url, headers=headers).content, "lxml")
# 下面这一步是关键：
pdf_link = bs.find('div', id = 'buttons').find_all('a')[1].attrs.get("onclick").replace('location.href=','')[1:-1]

In [59]:
response = requests.get('http:' + pdf_link, headers=headers)
with open(doi+'.pdf', 'wb') as f:
    f.write(response.content)